# On utilisera le truc etude n° 10

In [21]:
import numpy as np
import msicpe.san as san
from plotly import express as px
import scipy.signal as signal


H=30
Vvec_moy = [[] for i in range(5)]
lettres = ['a', 'e', 'i', 'o', 'u']
a = range(5)

for lettre,j in zip(lettres, a):
    signaux = []
    for i in range(5):
        dat = np.load(f'data_partie_2/apprentissage/{lettre}_{i+1}.npz') # dat est un dictionnaire contenant les données contenues dans le fichier .npz
        s = dat['s'] # extraction du signal utile
        t = dat['t'] # extraction du vecteur temps associé au signal

        nu, S = san.trans_fourier(s, t)
        S = np.abs(S)

        s_centre= s - np.mean(s)
        nu, S_norm = san.trans_fourier(s_centre, t)
        S_norm = np.abs(S_norm)
        S_norm /= np.max(S_norm)

        f0 = san.detect_fondamentale(S_norm, nu)

        # Liste des harmoniques
        freq_list = [f0 * k for k in range(1, H+1)]

        freq_peaks, amp_peaks = san.detect_pics(S, freq_list, nu)

        vec = np.zeros(H+1)
        vec[0] = np.log10(np.max(S))
        vec[1:] = np.log10(amp_peaks)
        signaux.append(vec)


    k = np.arange(H+1)
    vec_moy = np.mean(signaux, axis=0)
    Vvec_moy[j] = vec_moy

    k = list(range(H+1))
    indice = []
    ampl = []
    labels = []

    for i in range(5):
        indice.extend(k)
        ampl.extend(signaux[i])
        labels.extend([f"Son {i+1}"] * (H+1))

    indice.extend(k)
    ampl.extend(vec_moy)
    labels.extend(["Moyenne"] * (H+1))

    fig = px.line(x=indice, y=ampl, color=labels, markers=True,title=f"Voyelle: {lettre} - Spectres (log10) des 5 sons + vecteur moyen", labels={"x": "Indice d'harmonique k", "y": "Amplitude log10"}).show()

print(Vvec_moy)

[array([2.64557343, 2.62670417, 2.51545572, 1.93272867, 2.13913407,
       2.06125848, 2.32273543, 2.15781007, 2.08410504, 1.9197325 ,
       1.75826278, 2.0299171 , 1.54770949, 1.13119239, 0.84982708,
       0.6220214 , 0.50184461, 0.40352323, 0.51586616, 0.35892606,
       0.45526825, 0.55428196, 0.60658889, 0.6904648 , 0.90152448,
       0.78359516, 0.52144461, 0.53055702, 0.53905673, 0.57806164,
       0.56352012]), array([2.8533247 , 2.77255287, 2.77140889, 2.32289963, 2.59869447,
       1.83658841, 1.52414827, 1.44557743, 1.40972273, 1.15881951,
       1.36655216, 1.35733448, 1.02947407, 0.75386001, 0.84572014,
       0.64106956, 0.71476045, 0.66413012, 0.63366023, 0.62238366,
       0.59309982, 0.65639135, 0.50006818, 0.63258585, 0.69981022,
       0.62347793, 0.52543386, 0.5002509 , 0.53204492, 0.5455828 ,
       0.54188877]), array([ 3.06390924,  3.02719011,  2.91139307,  1.94860153,  1.21921098,
        0.69322566,  0.64766189,  0.41636877,  0.13138448,  0.06258026,
       -0

In [22]:
dat = np.load(f'data_partie_2/test/a_6.npz') # dat est un dictionnaire contenant les données contenues dans le fichier .npz
s = dat['s'] # extraction du signal utile
t = dat['t'] # extraction du vecteur temps associé au signal

s_centre= s - np.mean(s)
nu, S_norm = san.trans_fourier(s_centre, t)
S_norm = np.abs(S_norm)
S_norm /= np.max(S_norm)

f0 = san.detect_fondamentale(S_norm, nu)

# Liste des harmoniques
freq_list = [f0 * k for k in range(1, H+1)]

freq_peaks, amp_peaks = san.detect_pics(S, freq_list, nu)

vec = np.zeros(H+1)
vec[0] = np.log10(np.max(S))
vec[1:] = np.log10(amp_peaks)

def norme(s1,s2):
        return sum(abs(s1-s2))

L_norme = []
    
for i in range(len(lettres)):
    L_norme.append(norme(vec, Vvec_moy[i]))

print(L_norme)
print(min(L_norme))

[np.float64(29.8301574042067), np.float64(26.253303332254294), np.float64(12.383273997809738), np.float64(17.070261229341853), np.float64(15.007362780557015)]
12.383273997809738
